## 🧠 OpenCV 常见滤波方法对比总结

| 滤波类型             | 原理（无公式）                | 常用函数                                                  | 优点                     | 缺点                    | 应用场景                | 计算性能 / 速度分析                        |
| ---------------- | ---------------------- | ----------------------------------------------------- | ---------------------- | --------------------- | ------------------- | ---------------------------------- |
| 均值滤波 (Mean)      | 用邻域像素的平均值替代中心像素，线性平滑   | `cv2.blur(src, ksize)`                                | 实现简单；速度快；对轻度随机噪声有效     | 边缘易被模糊；细节损失；对椒盐噪声不理想  | 轻度去噪；整体平滑；快速模糊      | 复杂度约 N*k^2；对小核非常快；CPU/GPU均易加速      |
| 方框滤波 (Box)       | 与均值类似，可选择是否归一化；支持更灵活卷积 | `cv2.boxFilter(src, ddepth, ksize, normalize=True)`   | 可归一化或直接求和；核大时可用积分图加速   | 不归一化会偏亮；同样模糊边缘        | 模糊特效；快速积分滤波；卷积基础    | 积分图可达近似 O(N)；适合大核实时模糊              |
| 高斯滤波 (Gaussian)  | 邻域按距离加权平均，中心权重大，线性可分离  | `cv2.GaussianBlur(src, ksize, sigmaX)`                | 视觉平滑自然；抑制高频噪声；常作预处理    | 仍会模糊边缘；需调核和标准差        | 去高斯噪声；Canny等边缘检测前处理 | 可分离卷积将复杂度降至约 N*k；实时性好，GPU友好        |
| 中值滤波 (Median)    | 取邻域像素的中间值，非线性，抑制极端点    | `cv2.medianBlur(src, ksize)`                          | 去椒盐噪声效果好；保边缘；不易产生平均化模糊 | 需排序较慢；对连续噪声效果一般；大核会变形 | 医学影像去噪；椒盐噪声清除       | 复杂度约 N*k^2*log(k^2)；中大核明显变慢；更适合静态图 |
| 双边滤波 (Bilateral) | 同时考虑空间距离与像素差异，保边平滑     | `cv2.bilateralFilter(src, d, sigmaColor, sigmaSpace)` | 去噪同时保留边缘与细节；人像和HDR效果好  | 计算量大；参数敏感；不适合大图实时     | 人像磨皮；HDR去噪；细节保留的平滑  | 复杂度约 N*d^2；五者中最慢之一；可用降采样等近似加速      |

---
| 滤波类型                          | 原理（含公式与矩阵）                                                                                                                                                                                                              | 常用函数                                                | 优点                           | 缺点        | 应用场景                                      |                                 
| ----------------------------- | ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------- | ---------------------------- | --------- | ----------------------------------------- |
| **均值滤波 (Mean Filter)**        | 对图像进行卷积操作，用邻域像素的平均值替代中心像素。<br>公式：<br>$$g(x,y) = \frac{1}{m \times n} \sum_{i=-k}^{k} \sum_{j=-k}^{k} f(x+i, y+j)$$<br>卷积核示例（3×3）：<br>$$K = \frac{1}{9} \begin{bmatrix}1 & 1 & 1 \ 1 & 1 & 1 \ 1 & 1 & 1 \end{bmatrix}$$ | `cv2.blur(src, ksize)`                              | 实现简单，去除随机噪声                  | 模糊边缘，细节损失 | 图像平滑、模糊化预处理                               |                                 
| **方框滤波 (Box Filter)**         | 与均值滤波类似，但是否归一化可控：<br>$$g(x,y) = \frac{1}{N} \sum_{i,j \in \Omega} f(x+i, y+j)$$（当 normalize=True）                                                                                                                       | `cv2.boxFilter(src, ddepth, ksize, normalize=True)` | 灵活，可自定义归一化                   | 同样模糊边缘    | 卷积基础操作、快速模糊                               |                                 
| **高斯滤波 (Gaussian Filter)**    | 加权平均滤波，权值符合二维高斯分布：<br>$$G(x,y) = \frac{1}{2\pi\sigma^2} e^{-\frac{x^2 + y^2}{2\sigma^2}}$$<br>卷积结果：<br>$$g(x,y) = \sum_{i,j} G(i,j)f(x-i, y-j)$$                                                                        | `cv2.GaussianBlur(src, ksize, sigmaX)`              | 平滑自然，抑制高频噪声                  | 模糊边缘      | 高斯噪声去除、Canny 预处理                          |                                 
| **中值滤波 (Median Filter)**      | 非线性滤波，取邻域像素的中值：<br>$$g(x,y) = \text{median}{ f(x+i, y+j) (i,j) \in \Omega }$$                                | `cv2.medianBlur(src, ksize)` | 有效去除椒盐噪声  | 计算耗时，无法处理连续噪声                             | 医学图像、椒盐噪声场景                     |           
| **双边滤波 (Bilateral Filter)**   | 综合空间距离与像素值差异的加权平均：<br>$$g(p) = \frac{1}{W_p}\sum_{q \in \Omega} f(q) \cdot e^{-\frac{  p-q ^2}{2\sigma_s^2}} \cdot e^{-\frac{f(p)-f(q)^2}{2\sigma_r^2}}$$<br>其中：$$W_p$$ 为归一化系数 | `cv2.bilateralFilter(src, d, sigmaColor, sigmaSpace)` | 保边去噪，效果自然 | 计算量大 | 美颜、HDR、图像增强 |

---
| 滤波类型                          | 原理（含公式与矩阵）                                                                                                                                                                                                              | 常用函数                                                | 优点                           | 缺点        | 应用场景                                      |                                 
| ----------------------------- | ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------- | ---------------------------- | --------- | ----------------------------------------- |
| **自定义卷积滤波 (Custom Filter)**   | 利用自定义卷积核：<br>$$g(x,y) = \sum_{i=-k}^{k}\sum_{j=-k}^{k} f(x+i,y+j) \cdot h(i,j)$$<br>例如锐化核：<br>$$K = \begin{bmatrix} 0 & -1 & 0 \ -1 & 5 & -1 \ 0 & -1 & 0 \end{bmatrix}$$                                               | `cv2.filter2D(src, ddepth, kernel)`                 | 灵活，可实现任意滤波                   | 参数需人工设计   | 边缘增强、锐化、特效滤波                              |                                 
| **引导滤波 (Guided Filter)**      | 根据引导图 $I$ 实现局部线性平滑：<br>$$q_i = a_k I_i + b_k$$，其中 $$a_k, b_k$$ 通过最小化<br>$$E((a_k I_i + b_k - p_i)^2 + \epsilon a_k^2)$$ 得到<br>可近似为：$$q = \bar{a}I + \bar{b}$$                                                           | （OpenCV 无内置，需扩展）                                    | 高效保边，抗梯度反转                   | 实现复杂      | 深度图优化、人像磨皮                                |                                 
| **非局部均值滤波 (Non-Local Means)** | 基于块相似度的加权平均：<br>$$g(x) = \frac{1}{C(x)} \sum_{y} f(y) \cdot e^{-\frac{  P(x)-P(y)       ^2}{h^2}}$$<br>其中 $$P(x)$$ 为以 $x$ 为中心的邻域块 | `cv2.fastNlMeansDenoising(src)` | 去噪强、细节保留  | 非常耗时 | 高质量去噪、摄影后期                               |                                                      
| **高通滤波 (Laplacian Filter)**   | 计算二阶导数以突出边缘：<br>$$\nabla^2 f = \frac{\partial^2 f}{\partial x^2} + \frac{\partial^2 f}{\partial y^2}$$<br>常用卷积核：<br>$$K = \begin{bmatrix} 0 & 1 & 0 \ 1 & -4 & 1 \ 0 & 1 & 0 \end{bmatrix}$$                            | `cv2.Laplacian(src, ddepth)`                        | 突出边缘细节                       | 放大噪声      | 边缘检测、锐化                                   |                                 
| **Sobel / Scharr 滤波**         | 近似计算一阶导数：<br>$$G_x = \begin{bmatrix} -1 & 0 & 1 \ -2 & 0 & 2 \ -1 & 0 & 1 \end{bmatrix} , \quad G_y = G_x^T$$<br>梯度幅值：$$G = \sqrt{G_x^2 + G_y^2}$$                                                                      | `cv2.Sobel(src, ddepth, dx, dy)` / `cv2.Scharr()`   | 对噪声鲁棒，边缘方向可控                 | 不保边，需预平滑  | 边缘提取、特征检测                                 |                                 


```
┌──────────────────────────────────────────────┐
│ 原图 │ 均值滤波 │ 方框滤波 │
│----------------------------------------------│
│ 中值滤波 │ 高斯滤波 │ 双边滤波 │
└──────────────────────────────────────────────┘

```

In [1]:
import cv2
import numpy as np

# 1️⃣ 读取图像
img = cv2.imread("../image/color_noise.png")
if img is None:
    raise ValueError("⚠️ 未找到图像，请检查路径！")

# 调整尺寸方便展示
img = cv2.resize(img, (600, 400))

# ============================================
# 2️⃣ 各种滤波操作与展示
# ============================================

# 【1】均值滤波（Mean Filter）
# 原理：用邻域内所有像素的平均值替换中心像素，达到平滑效果
# 缺点：模糊边缘，容易丢失细节
blur = cv2.blur(img, (5, 5))
# cv2.imshow("1️⃣ Mean Filter (均值滤波)", blur)

# 【2】方框滤波（Box Filter）
# 原理：与均值滤波类似，但可控制是否归一化（normalize 参数）
# normalize=True 表示与均值滤波相同；False 表示直接累加
box = cv2.boxFilter(img, -1, (5, 5), normalize=True)
# cv2.imshow("2️⃣ Box Filter (方框滤波)", box)

# 【3】中值滤波（Median Filter）
# 原理：取邻域像素的中位数作为输出，能有效去除椒盐噪声
# 特点：非线性滤波，保边效果好
median = cv2.medianBlur(img, 5)
# cv2.imshow("3️⃣ Median Filter (中值滤波)", median)

# 【4】高斯滤波（Gaussian Filter）
# 原理：邻域像素按高斯权重加权平均，距离中心越近权重越大
# σ（sigmaX）控制平滑程度
gaussian = cv2.GaussianBlur(img, (5, 5), 1.5)
# cv2.imshow("4️⃣ Gaussian Filter (高斯滤波)", gaussian)

# 【5】双边滤波（Bilateral Filter）
# 原理：结合空间距离与像素差异的双重权重，既去噪又保留边缘
# d：邻域直径；sigmaColor：颜色差异权重；sigmaSpace：空间距离权重
bilateral = cv2.bilateralFilter(img, d=9, sigmaColor=75, sigmaSpace=75)
# cv2.imshow("5️⃣ Bilateral Filter (双边滤波)", bilateral)

# ============================================
# 3️⃣ 合并展示（便于对比）
# ============================================

# 第一行：原图、均值、方框
top_row = np.hstack((img, blur, box))

# 第二行：中值、高斯、双边
bottom_row = np.hstack((median, gaussian, bilateral))

# 空白分隔区域
blank = np.zeros((50, top_row.shape[1], 3), np.uint8)

# 合并拼接
combined = np.vstack((top_row, blank, bottom_row))

# 统一显示结果
cv2.imshow("📊 Filter Comparison (Original, Mean, Box | Median, Gaussian, Bilateral)", combined)

# 等待用户按键关闭
cv2.waitKey(0)
cv2.destroyAllWindows()


In [2]:
# 均值滤波
import cv2

# 1.读取要滤波的图像
image_np = cv2.imread("../image/image.jpg")

# 2.获取卷积核并进行均值滤波
# 参数说明：
#   src : 输入图像（numpy.ndarray），可以是灰度或彩色图。
#   ksize : 卷积核大小（tuple类型），格式为 (width, height)，
#           例如 (3, 3) 表示使用 3×3 的邻域窗口。
#           核越大，模糊越明显，但细节损失也越多。
#   dst : （可选）输出结果数组，默认 None，表示函数返回新图像。
#   anchor : 锚点位置，默认 (-1, -1)，表示锚点位于卷积核中心。
#   borderType : 边界处理方式，默认 cv2.BORDER_DEFAULT。
#                其他常用选项：
#                - cv2.BORDER_CONSTANT：常数填充
#                - cv2.BORDER_REPLICATE：复制边界像素
#                - cv2.BORDER_REFLECT：边界镜像
image_blur = cv2.blur(image_np, (3, 3))

# 3.显示
cv2.imshow("image_blur", image_blur)
cv2.waitKey(0)

-1

In [4]:
# 方框滤波
import cv2

# 1.读取要滤波的图像
image_np = cv2.imread("../image/image.jpg")

# 在使用 cv2.boxFilter() 时，如果参数 normalize=True（默认值），那么它的行为就完全等价于 cv2.blur()
# 参数说明：
# ------------------------------------------------------------
# src : 输入图像（numpy.ndarray），可以是灰度图或彩色图。
#
# ddepth : 输出图像的深度（数据类型）。
#           - 设置为 -1 表示与输入图像保持一致（最常用）。
#           - 也可指定 cv2.CV_8U, cv2.CV_32F 等类型。
#
# ksize : 卷积核大小（tuple 类型），格式为 (width, height)。
#          - (3, 3) 表示使用 3×3 的窗口进行滤波。
#          - 数值越大，平滑效果越强，但边缘越模糊。
#
# dst : （可选）输出图像的存储对象，一般不填（默认返回结果）。
#
# anchor : 锚点位置（卷积核的中心）。
#           - 默认为 (-1, -1)，表示核中心自动定位在中间。
#           - 若需要自定义锚点位置，可传入 (x, y) 坐标。
#
# normalize : 是否对卷积核归一化。
#           - True（默认） → 每个像素取邻域平均值（均值滤波效果）
#           - False → 直接累加邻域像素值（亮度明显提升）
#
# borderType : 边界填充方式（默认 cv2.BORDER_DEFAULT）。
#           - cv2.BORDER_CONSTANT ：常数填充
#           - cv2.BORDER_REPLICATE ：复制边界像素
#           - cv2.BORDER_REFLECT ：镜像填充
#           - cv2.BORDER_WRAP ：环绕填充
#
# 对比关系：
# ------------------------------------------------------------
# 当 normalize=True 时：
#   cv2.boxFilter(src, -1, (k, k), normalize=True)
#       ≡ cv2.blur(src, (k, k))
#
# 当 normalize=False 时：
#   cv2.boxFilter(src, -1, (k, k), normalize=False)
#       → 不做平均，结果数值更高、亮度提升。
# ------------------------------------------------------------
image_box = cv2.boxFilter(image_np, -1, (3, 3))

# 3.显示
cv2.imshow("image_box", image_box)
cv2.waitKey(0)

-1

In [ ]:
# 高斯滤波
import cv2

# 1.读取要滤波的图像
image_np = cv2.imread("../image/image.jpg")

# 2.高斯滤波
# 函数原型：
#   cv2.GaussianBlur(src, ksize, sigmaX[, dst[, sigmaY[, borderType]]])
#
# 作用：
#   对图像进行高斯平滑（Gaussian Smoothing），
#   通过高斯加权平均的方式降低噪声并平滑图像。
#   与均值滤波不同，高斯滤波对距离中心较近的像素赋予更高权重，
#   能有效抑制高频噪声，同时保持整体自然的模糊效果。
# 参数说明：
# ------------------------------------------------------------
# src : 输入图像（numpy.ndarray），可以是灰度或彩色图。
#
# ksize : 高斯核大小（tuple 类型），格式为 (width, height)。
#          - 必须为奇数，如 (3,3)、(5,5)、(7,7)。
#          - 核越大，模糊越强。
#
# sigmaX : X方向的标准差（σ）。
#           - 控制水平方向的平滑程度。
#           - 值越大，模糊越明显。
#           - 若为 0，OpenCV 会根据核大小自动计算。
#
# dst : （可选）输出图像存储对象，一般不使用。
#
# sigmaY : Y方向的标准差。
#           - 若设为 0，表示与 sigmaX 相同。
#           - 可以独立控制垂直方向模糊。
#
# borderType : 边界扩展方式，默认 cv2.BORDER_DEFAULT。
#           - cv2.BORDER_CONSTANT ：常数填充
#           - cv2.BORDER_REPLICATE ：复制边缘像素
#           - cv2.BORDER_REFLECT ：镜像填充
# ------------------------------------------------------------
image_gus = cv2.GaussianBlur(image_np, (3, 3), 1)

# 3.显示
cv2.imshow("image_gus", image_gus)
cv2.waitKey(0)

In [5]:
# 中值滤波
import cv2

# 1.读取要滤波的图像
image_np = cv2.imread("../image/image.jpg")

# 中值滤波（Median Filter）
# ------------------------------------------------------------
# 函数原型：
#   cv2.medianBlur(src, ksize)
#
# 作用：
#   对图像进行中值滤波（Median Filtering）。
#   它不是取邻域像素的平均值，而是取邻域像素的“中间值”作为输出。
#   这是一种 **非线性滤波**，能在保留边缘的同时有效去除“椒盐噪声”。
# 参数说明：
# ------------------------------------------------------------
# src : 输入图像（numpy.ndarray）
#       可以是灰度图或彩色图（BGR），但必须为单通道或三通道 8位图像。
#
# ksize : 滤波核大小（int 类型，必须为奇数）
#          - 表示窗口的宽度和高度（如 3、5、7）。
#          - 越大，平滑效果越强，但细节丢失越多。
#
# ⚠️ 注意：
#   - ksize 必须是奇数，且大于 1。
#   - 不支持浮点图像（如 CV_32F），仅支持 8 位或 16 位整型图像。
# ------------------------------------------------------------
image_mid = cv2.medianBlur(image_np,1)
# 3.显示
cv2.imshow("image_mid", image_mid)
cv2.waitKey(0)

-1

In [8]:
# 双边滤波
import cv2

# 1.读取要滤波的图像
image_np = cv2.imread("../image/image.jpg")
# 双边滤波（Bilateral Filter）
# ------------------------------------------------------------
# 函数原型：
#   cv2.bilateralFilter(src, d, sigmaColor, sigmaSpace[, dst[, borderType]])
#
# 作用：
#   双边滤波是一种 **边缘保留型的非线性滤波器**。
#   它结合了空间距离与像素值差异两种权重，
#   在去除噪声的同时尽可能保留图像边缘（不模糊轮廓）。
# 参数说明：
# ------------------------------------------------------------
# src : 输入图像（numpy.ndarray）
#       可以是灰度图或彩色图（BGR 格式）。
#
# d : 邻域直径（int）
#     - 表示用于滤波的邻域像素直径。
#     - 如果设为非正值（<=0），OpenCV 会根据 sigmaSpace 自动计算。
#     - 一般推荐范围：5 ~ 15。
#
# sigmaColor : 颜色空间标准差（float）
#     - 表示像素值差异的容忍度。
#     - 值越大，更多不同颜色的像素会混合在一起（更模糊）。
#     - 值越小，只会平滑相似颜色的区域（保边更明显）。
#
# sigmaSpace : 空间距离标准差（float）
#     - 表示空间（几何位置）影响的范围。
#     - 值越大，邻域范围越广。
#     - 值越小，滤波仅影响局部区域。
#
# dst : （可选）输出图像，一般省略。
#
# borderType : 边界处理方式（默认 cv2.BORDER_DEFAULT）
# ------------------------------------------------------------
imag_bilateral = cv2.bilateralFilter(image_np,5,75,75)
# 3.显示
cv2.imshow("image_mid", image_mid)
cv2.waitKey(0)

-1

| 滤波类型                        | OpenCV 函数                                             | 参数名          | 参数说明                                                  | 典型用法与建议             |
| --------------------------- | ----------------------------------------------------- | ------------ | ----------------------------------------------------- | ------------------- |
| **均值滤波** (Mean Filter)      | `cv2.blur(src, ksize, dst=None)`                      | `src`        | 输入图像（numpy.ndarray）                                   | 必填                  |
|                             |                                                       | `ksize`      | 滤波核大小（tuple类型，如 `(5, 5)`），表示卷积窗口的宽与高                  | 数值越大，平滑效果越强，但细节丢失越多 |
|                             |                                                       | `dst`        | 可选参数，输出结果存储对象（一般省略）                                   | 一般不使用               |
|                             |                                                       | **说明**       | 对图像进行均值卷积操作，将邻域内像素的平均值作为输出                            | 简单快速，适合去除随机噪声       |
| **方框滤波** (Box Filter)       | `cv2.boxFilter(src, ddepth, ksize, normalize=True)`   | `src`        | 输入图像                                                  | 必填                  |
|                             |                                                       | `ddepth`     | 输出图像深度，设为 `-1` 表示与原图一致                                | 常用 `-1`             |
|                             |                                                       | `ksize`      | 卷积核大小，如 `(5, 5)`                                      | 影响模糊程度              |
|                             |                                                       | `normalize`  | 是否归一化（布尔类型）<br>`True` → 与均值滤波相同；<br>`False` → 直接累加不平均 | 通常设为 `True`         |
|                             |                                                       | **说明**       | 与均值滤波类似，但提供更灵活的控制                                     | 可模拟不同模糊强度           |
| **中值滤波** (Median Filter)    | `cv2.medianBlur(src, ksize)`                          | `src`        | 输入图像                                                  | 必填                  |
|                             |                                                       | `ksize`      | 滤波核大小（奇数整数，如 3、5、7...）                                | 必须为奇数               |
|                             |                                                       | **说明**       | 使用邻域像素的中值代替中心像素值，属于非线性滤波                              | 对椒盐噪声效果极好，边缘保留能力强   |
| **高斯滤波** (Gaussian Filter)  | `cv2.GaussianBlur(src, ksize, sigmaX, sigmaY=0)`      | `src`        | 输入图像                                                  | 必填                  |
|                             |                                                       | `ksize`      | 高斯核大小，如 `(5, 5)`（必须为奇数）                               | 控制模糊程度              |
|                             |                                                       | `sigmaX`     | X方向高斯标准差（σ）                                           | 控制平滑强度              |
|                             |                                                       | `sigmaY`     | Y方向标准差（可选，默认0表示与X相同）                                  | 通常不修改               |
|                             |                                                       | **说明**       | 按高斯分布权重对邻域像素加权平均，距中心越近权重越高                            | 平滑自然，适合去除高斯噪声       |
| **双边滤波** (Bilateral Filter) | `cv2.bilateralFilter(src, d, sigmaColor, sigmaSpace)` | `src`        | 输入图像                                                  | 必填                  |
|                             |                                                       | `d`          | 邻域直径（像素单位）。过小会欠滤波，过大会导致模糊                             | 一般 5～15             |
|                             |                                                       | `sigmaColor` | 颜色空间标准差。数值越大，考虑的颜色差异越多                                | 控制颜色平滑程度            |
|                             |                                                       | `sigmaSpace` | 空间距离标准差。数值越大，影响范围越广                                   | 控制空间平滑范围            |
|                             |                                                       | **说明**       | 同时考虑**空间距离**与**像素差异**的加权平均。既能去噪，又能保留边缘                | 图像美化、人像磨皮、HDR等常用    |


### 🧠 图像噪声类型总结表
| 噪声类型                                      | 特点描述                      | 视觉表现            | 常见来源                | 推荐去噪方法                                                      |
| ----------------------------------------- | ------------------------- | --------------- | ------------------- | ----------------------------------------------------------- |
| **高斯噪声 (Gaussian Noise)**                 | 噪声服从正态分布，随机连续分布在均值附近，整体均匀 | 图像呈现轻微颗粒感，整体偏灰  | 传感器热噪声、光照不足、图像传输误差  | ✅ 高斯滤波（`cv2.GaussianBlur`）<br>✅ 双边滤波（`cv2.bilateralFilter`） |
| **椒盐噪声 (Salt-and-Pepper Noise)**          | 部分像素随机变为黑色或白色，属于脉冲型噪声     | 图像出现散布的黑白点      | 传感器故障、数据传输错误、压缩损坏   | ✅ 中值滤波（`cv2.medianBlur`）<br>✅ 自适应中值滤波                       |
| **泊松噪声 (Poisson / Shot Noise)**           | 与信号强度相关，亮处噪声更多，暗处噪声较少     | 高亮区域噪声明显、暗区较干净  | 光子统计波动、低光照环境拍摄      | ✅ 维纳滤波<br>✅ 自适应滤波                                           |
| **乘性噪声 (Multiplicative / Speckle Noise)** | 噪声与像素亮度成正比，亮的地方受影响更大      | 图像呈现斑点状纹理、亮暗波动  | 雷达、超声波、SAR 成像       | ✅ 各向异性扩散滤波<br>✅ 自适应滤波                                       |
| **伪随机噪声 (Uniform Noise)**                 | 噪声强度在一定范围内均匀分布            | 图像整体灰度随机波动      | 模拟干扰、量化误差、传感器不稳定    | ✅ 均值滤波<br>✅ 高斯滤波                                            |
| **量化噪声 (Quantization Noise)**             | 图像离散化或压缩导致灰度跳变、失真         | 图像出现色带、分层现象     | 图像压缩（JPEG）、A/D 转换误差 | ✅ 双边滤波<br>✅ 非局部均值滤波                                         |
| **周期性噪声 (Periodic Noise)**                | 噪声呈周期性重复，通常表现为正弦波或条纹      | 图像出现规则条纹或干涉花纹   | 电磁干扰、机械震动、扫描误差      | ✅ 频域带阻滤波器<br>✅ 傅里叶变换去噪                                      |
| **脉冲噪声 (Impulse Noise)**                  | 像素值随机突变（非固定黑白值）           | 图像中出现少量极亮或极暗像素点 | 传输误码、传感器抖动          | ✅ 中值滤波<br>✅ 自适应滤波                                           |


### 🧮 OpenCV 常见滤波方式分类与中文原理说明
| 分类                               | 滤波类型                          | 常用函数                                                  | 原理说明（中文解释）                                               | 主要特征            | 优缺点                              | 常见应用              |
| -------------------------------- | ----------------------------- | ----------------------------------------------------- | -------------------------------------------------------- | --------------- | -------------------------------- | ----------------- |
| **线性滤波 (Linear Filtering)**      | **均值滤波 (Mean Filter)**        | `cv2.blur(src, ksize)`                                | 以每个像素为中心，取其邻域内所有像素的平均值，用平均结果替代当前像素，达到整体平滑效果。             | 对噪声有一定抑制作用，计算简单 | 优点：速度快、实现容易；缺点：边缘模糊、细节丢失         | 图像平滑、轻度去噪、预处理     |
|                                  | **方框滤波 (Box Filter)**         | `cv2.boxFilter(src, ddepth, ksize, normalize=True)`   | 与均值滤波类似，但可控制是否进行归一化。归一化时效果与均值滤波相同，不归一化时会直接累加像素值，可实现亮度增强。 | 可快速处理大核卷积       | 优点：灵活可控、效率高；缺点：同样模糊边缘            | 模糊特效、积分图滤波、卷积模板计算 |
|                                  | **高斯滤波 (Gaussian Filter)**    | `cv2.GaussianBlur(src, ksize, sigmaX)`                | 根据高斯分布为邻域像素赋予不同权重，距离中心越近的像素权重越大，使得平滑更自然，噪声被抑制而主轮廓保留。     | 权值呈正态分布，中心权重大   | 优点：平滑自然、视觉柔和；缺点：仍会模糊边缘           | 高斯噪声去除、边缘检测前处理    |
| **非线性滤波 (Non-Linear Filtering)** | **中值滤波 (Median Filter)**      | `cv2.medianBlur(src, ksize)`                          | 对邻域内像素值进行排序，取中间值作为输出。能有效去除孤立的亮点或暗点（椒盐噪声），并保持边缘清晰。        | 非线性排序滤波         | 优点：去椒盐噪声效果好、保边缘；缺点：计算慢、对高斯噪声效果一般 | 医学图像、黑白点噪声去除      |
|                                  | **双边滤波 (Bilateral Filter)**   | `cv2.bilateralFilter(src, d, sigmaColor, sigmaSpace)` | 结合空间距离与像素值差异，对颜色相近且空间距离近的像素赋予更高权重，实现“去噪同时保边”。            | 空间与颜色双权重滤波      | 优点：保边效果极佳、视觉自然；缺点：计算量大、参数敏感      | 人像美颜、HDR增强、图像细节优化 |
|                                  | **非局部均值滤波 (Non-Local Means)** | `cv2.fastNlMeansDenoising(src)`                       | 比较整块像素区域之间的相似性，相似块的像素对当前像素影响更大，实现全局加权去噪，保留更多纹理细节。        | 全局相似性加权滤波       | 优点：去噪能力强、细节保持好；缺点：速度慢、计算复杂       | 高质量图像去噪、照片修复      |
|                                  | **自适应滤波 (Adaptive Filter)**   | 自定义实现                                                 | 根据图像的局部统计特征（均值与方差）动态调整滤波强度。噪声多的区域平滑更强，纹理丰富区域平滑较弱。        | 自适应平滑滤波         | 优点：保边能力强、适应性好；缺点：实现复杂、计算开销大      | 医学图像、雷达噪声、自适应去噪   |
